In [1]:
import os
import os.path as osp
import argparse
from datetime import date
import json
import random
import time
from pathlib import Path
import numpy as np
import numpy.linalg as LA
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import csv
import warnings

import skimage.io
import skimage.transform

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torch

import datasets
import util.misc as utils
from config import cfg
import h5py

/home/kmuvcl/anaconda3/envs/debug/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def eul2rotm_ypr(euler):
    R_x = np.array(
        [
            [1, 0, 0],
            [0, np.cos(euler[0]), -np.sin(euler[0])],
            [0, np.sin(euler[0]), np.cos(euler[0])],
        ],
        dtype=np.float32,
    )

    R_y = np.array(
        [
            [np.cos(euler[1]), 0, np.sin(euler[1])],
            [0, 1, 0],
            [-np.sin(euler[1]), 0, np.cos(euler[1])],
        ],
        dtype=np.float32,
    )

    R_z = np.array(
        [
            [np.cos(euler[2]), -np.sin(euler[2]), 0],
            [np.sin(euler[2]), np.cos(euler[2]), 0],
            [0, 0, 1],
        ],
        dtype=np.float32,
    )

    return np.dot(R_z, np.dot(R_x, R_y))


def create_masks(image):
    masks = torch.zeros((1, height, width), dtype=torch.uint8)
    return masks


def read_line_file(filename, min_line_length=10):
    segs = []  # line segments
    # csv 파일 열어서 Line 정보 가져오기
    with open(filename, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            segs.append([float(row[0]), float(row[1]), float(row[2]), float(row[3])])
    segs = np.array(segs, dtype=np.float32)
    lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=1)
    segs = segs[lengths > min_line_length]
    return segs

def coordinate_yup(segs,org_h,org_w):
    H = np.array([0,org_h,0,org_h])
    segs[:,1] = -segs[:,1]
    segs[:,3] = -segs[:,3]
    # segs = -segs
    return(H+segs)
    

def normalize_segs(segs, pp, rho):
    pp = np.array([pp[0], pp[1], pp[0], pp[1]], dtype=np.float32)
    return (segs - pp)/rho

def focal_length_normalize(segs):
    segs = segs/517.97
    return segs


def normalize_safe_np(v, axis=-1, eps=1e-6):
    de = LA.norm(v, axis=axis, keepdims=True)
    de = np.maximum(de, eps)
    return v / de


def segs2lines_np(segs):
    ones = np.ones(len(segs))
    ones = np.expand_dims(ones, axis=-1)
    p1 = np.concatenate([segs[:, :2], ones], axis=-1)
    p2 = np.concatenate([segs[:, 2:], ones], axis=-1)

    lines = np.cross(p1, p2)

    return normalize_safe_np(lines)



def sample_segs_np(segs, num_sample, use_prob=True):
    num_segs = len(segs)
    sampled_segs = np.zeros([num_sample, 4], dtype=np.float32)
    mask = np.zeros([num_sample, 1], dtype=np.float32)
    if num_sample > num_segs:
        sampled_segs[:num_segs] = segs
        mask[:num_segs] = np.ones([num_segs, 1], dtype=np.float32)
    # else:
    #     sampled_segs = segs[:num_sample]
    #     mask[:num_segs] = np.ones([num_sample, 1], dtype=np.float32)
    else:
        lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=-1)
        prob = lengths / np.sum(lengths)
        idxs = np.random.choice(segs.shape[0], num_sample, replace=True, p=prob)
        sampled_segs = segs[idxs]
        mask = np.ones([num_sample, 1], dtype=np.float32)
    return sampled_segs, mask

# def segs_np(segs, num_sample, use_prob=True):
#     num_segs = len(segs)
#     sampled_segs = np.zeros([num_sample, 4], dtype=np.float32)
#     mask = np.zeros([num_sample, 1], dtype=np.float32)
#     if num_sample > num_segs:
#         sampled_segs[:num_segs] = segs
#         mask[:num_segs] = np.ones([num_segs, 1], dtype=np.float32)
#     # else:
#     #     sampled_segs = segs[:num_sample]
#     #     mask[:num_segs] = np.ones([num_sample, 1], dtype=np.float32)
#     else:
#         lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=-1)
#         prob = lengths / np.sum(lengths)
#         idxs = np.random.choice(segs.shape[0], num_sample, replace=True, p=prob)
#         sampled_segs = segs[idxs]
#         mask = np.ones([num_sample, 1], dtype=np.float32)
#     return sampled_segs, mask

def straight_line(segs):

    # for i in range(len(segs)):
    #     seg = np.vstack([segs[i,2]-segs[i,0],segs[i,3]-segs[i,1]])
    num_line = segs.shape[0]
    seg = np.zeros([num_line,2])
    for i in range(len(segs)):    
        seg[i,0] = segs[i,2]-segs[i,0]
        seg[i,1] = segs[i,3]-segs[i,1]
    return seg


def sample_vert_segs_np(segs, thresh_theta=22.5):
    lines = segs2lines_np(segs)
    (a, b) = lines[:, 0], lines[:, 1]
    theta = np.arctan2(np.abs(b), np.abs(a))
    thresh_theta = np.radians(thresh_theta)
    return segs[theta < thresh_theta]

def compute_vp_similarity(lines,vp):
    lines = torch.tensor(lines)
    vp = torch.tensor(vp)
    cos_sim = TF.cosine_similarity(lines, vp.unsqueeze(0), dim=-1).abs()
    
    return cos_sim

In [3]:
def load_h5py_to_dict(file_path):
        with h5py.File(file_path, 'r') as f:
            return {key: torch.tensor(f[key][:]) for key in f.keys()}

In [4]:
line_segments = []

In [93]:
img = "/home/kmuvcl/dataset/data/su3/000/0050_0.png"
h5_path = "/home/kmuvcl/dataset/data/su3/000/0052_0_line.h5py"
npz_file = "/home/kmuvcl/dataset/data/su3/000/0052_0_camera.json"
npz_file2 = "/home/kmuvcl/dataset/data/su3/000/0050_0_label.npz"
line_file = "/home/kmuvcl/dataset/data/su3/000/0052_0.jlk"

In [94]:
with np.load(npz_file2) as npz:
        npz_content = npz['vpts']

EOFError: No data left in file

In [95]:
npz_content

array([[-0.88214494,  0.15882524,  0.44339018],
       [-0.47097803, -0.29748071, -0.83047271],
       [ 0.        ,  0.94142433, -0.3372243 ]])

In [54]:
with open(line_file, 'r') as file:
    for line in file:
        # 줄바꿈 문자 제거 및 공백으로 구분
        parts = line.strip().split()

        # parts 리스트에 5개의 요소가 있는지 확인 (x1, y1, x2, y2, type)
        if len(parts) == 5:
            # 좌표와 타입을 float 또는 적절한 타입으로 변환
            x1, y1, x2, y2, segment_type = map(float, parts)

            # 변환된 데이터를 리스트에 추가
            line_segments.append([x1, y1, x2, y2])

In [55]:
line_segments

[[389.416, 279.794, 366.91, 278.677],
 [402.034, 345.944, 363.341, 339.354],
 [435.823, 351.968, 404.421, 346.604],
 [509.434, 359.401, 456.815, 356.862],
 [402.301, 410.273, 364.483, 400.219],
 [488.177, 430.142, 460.461, 427.129],
 [365.525, 287.94, 452.958, 296.149],
 [434.741, 419.274, 403.953, 410.932],
 [273.387, 431.178, 254.488, 424.074],
 [366.367, 471.947, 425.087, 494.525],
 [353.596, 381.856, 353.164, 426.875],
 [387.187, 468.492, 364.433, 460.459],
 [453.084, 296.586, 509.498, 299.009],
 [471.798, 440.784, 509.459, 442.854],
 [480.563, 370.165, 509.438, 371.036],
 [353.18, 433.113, 352.717, 479.383],
 [364.048, 410.513, 469.322, 440.808],
 [252.273, 423.369, 224.873, 412.526],
 [352.667, 429.406, 333.491, 436.601],
 [365.309, 349.812, 479.274, 371.165],
 [354.445, 274.366, 353.65, 376.869],
 [252.339, 473.422, 225.711, 460.449],
 [436.907, 283.974, 408.141, 281.678],
 [509.435, 430.376, 489.217, 429.564],
 [352.869, 377.197, 333.323, 382.591],
 [352.923, 485.591, 352.361, 

In [56]:
dict = np.load(npz_file2)

In [57]:
dict.keys()


KeysView(NpzFile '/home/kmuvcl/dataset/data/su3/000/0001_0_label.npz' with keys: vpts)

In [87]:
with open(npz_file) as f:
    js = json.load(f)
    RT = np.array(js["modelview_matrix"])

vpts = []
for axis in [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]]:
    vp = RT @ axis
    vp = np.array([vp[0], vp[1], -vp[2]])
    vp /= LA.norm(vp)
    if vp[2] < 0.0: vp *= -1.0
    vpts.append(vp)

In [88]:
vpts = torch.tensor(vpts)

In [89]:
vpts

tensor([[-0.6796,  0.0339,  0.7328],
        [ 0.7336,  0.0314,  0.6789],
        [-0.0000, -0.9989,  0.0462]], dtype=torch.float64)

In [61]:
torch.dot(vpts[1],vpts[2])

tensor(0., dtype=torch.float64)

In [62]:
image = skimage.io.imread(img)[:, :, 0:3]

In [63]:
image.shape

(512, 512, 3)

In [64]:
dic = load_h5py_to_dict(h5_path)

In [65]:
keylines = dic['klines']
num_segs = keylines.shape[0]

In [66]:
org_segs = np.copy(keylines.reshape(num_segs,-1).numpy())

In [67]:
org_segs.shape

(250, 4)

In [72]:
org_segs

array([[291.79550171, 510.62802124, 292.5355835 , 224.37347412],
       [326.89810181, 508.77035522, 328.72384644, 226.2336731 ],
       [352.57461548, 508.77096558, 354.1239624 , 226.25204468],
       [327.28833008, 510.6350708 , 328.98913574, 229.37266541],
       [306.61767578, 510.62908936, 307.7722168 , 229.37355042],
       [306.37161255, 508.76071167, 307.50668335, 231.24491882],
       [291.91964722, 508.75805664, 292.46536255, 293.74673462],
       [353.67324829, 378.12341309, 354.01657104, 226.88270569],
       [358.75698853, 222.15316772, 508.78671265, 220.9914856 ],
       [365.49993896, 288.4559021 , 510.7774353 , 299.94802856],
       [365.56079102, 350.53616333, 508.77960205, 373.56582642],
       [366.09564209, 288.23898315, 508.8996582 , 299.32180786],
       [330.61990356, 222.59237671, 470.64297485, 221.2504425 ],
       [207.67799377, 228.76742554, 209.8717804 , 363.69110107],
       [365.32519531, 349.72930908, 479.2996521 , 371.02819824],
       [363.79342651, 470

In [73]:
line_segments

tensor([[389.4160, 279.7940, 366.9100, 278.6770],
        [402.0340, 345.9440, 363.3410, 339.3540],
        [435.8230, 351.9680, 404.4210, 346.6040],
        ...,
        [334.3750, 449.3750, 334.3750, 473.1250],
        [335.6250, 288.1250, 335.6250, 311.8750],
        [335.6250, 228.1250, 335.6250, 256.8750]])

In [69]:
line_segments = torch.tensor(line_segments)

In [70]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[0, 512, 512, 0])
for i in range(num_segs):
    plt.plot(
        (org_segs[i, 0], org_segs[i, 2]),
        (org_segs[i, 1], org_segs[i, 3]),
        c="r",
        alpha=1.0,
    )                                 
plt.xlim(0, 512)
plt.ylim(512, 0)
plt.axis('off')
plt.savefig('su3_LSD.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

In [71]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[0, 512, 512, 0])
for i in range(len(line_segments)):
    plt.plot(
        (line_segments[i, 0], line_segments[i, 2]),
        (line_segments[i, 1], line_segments[i, 3]),
        c="r",
        alpha=1.0,
    )                                 
plt.xlim(0, 512)
plt.ylim(512, 0)
plt.axis('off')
plt.savefig('su3.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

In [43]:
focal_length = 2.1875 * 256

In [ ]:
vpts_x = vpts[0]
vpts_y = vpts[1]
vpts_z = vpts[2]

In [ ]:
vpts_x = vpts_x/vpts_x[2] #* focal_length
vpts_y = vpts_y/vpts_y[2] #* focal_length 
vpts_z = vpts_z/vpts_z[2] #* focal_length

In [ ]:
vpts_x

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[-256/focal_length, 256/focal_length, 256/focal_length, -256/focal_length])
for i in range(num_segs):
    plt.plot((0, vpts_x[0]), (0, vpts_x[1]), 'r-', alpha=1.0)
    plt.plot((0, vpts_y[0]), (0, vpts_y[1]), 'g-', alpha=1.0)
    plt.plot((0, vpts_z[0]), (0, vpts_z[1]), 'b-', alpha=1.0)                               
plt.xlim(-256/focal_length, 256/focal_length)
plt.ylim(256/focal_length, -256/focal_length)
plt.axis('off')
plt.savefig('su3_vp.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

##YUD_dataset visualize

In [2]:
import os
import json
import math
import random
import glob
import numpy as np
import torch
import numpy.linalg as LA


import skimage.transform
import skimage.io

from scipy import io
import scipy.spatial.distance as scipy_spatial_dist

In [193]:
camera_params = io.loadmat(os.path.join("/home/kmuvcl/dataset/data/processed_data/YorkUrbanDB", "cameraParameters.mat"))
f = camera_params['focal'][0, 0]
ps = camera_params['pixelSize'][0, 0]
pp = camera_params['pp'][0, :]
K = np.matrix([[f / ps, 0, pp[0]], [0, f / ps, pp[1]], [0, 0, 1]])

In [196]:
ps

0.00896875

In [194]:
f

6.053170589753693

In [195]:
pp

array([307.55130528, 251.45424496])

In [49]:
ortho_vp = io.loadmat(os.path.join("/home/kmuvcl/dataset/data/processed_data/YorkUrbanDB/P1020817", "P1020817GroundTruthVP_Orthogonal_CamParams.mat"))

In [50]:
ortho_vp = torch.tensor(ortho_vp['vp_orthogonal'])

In [51]:
mat_lines = io.loadmat(os.path.join("/home/kmuvcl/dataset/data/processed_data/YorkUrbanDB/P1020817", "P1020817LinesAndVP.mat"))

In [52]:
len(mat_lines['lines'])

138

In [53]:
mat_lines = mat_lines['lines'].reshape(-1,4)

In [54]:
mat_lines.shape

(69, 4)

In [55]:
org_segs = torch.tensor(mat_lines/1.)

In [56]:
num_segs = len(org_segs)

In [57]:
org_segs.shape

torch.Size([69, 4])

In [58]:
manhattan_db = io.loadmat(os.path.join("/home/kmuvcl/dataset/data/processed_data/YorkUrbanDB", "Manhattan_Image_DB_Names.mat"))

In [59]:
len(manhattan_db['Manhattan_Image_DB_Names'])

102

In [60]:
iname = os.path.join("/home/kmuvcl/dataset/data/processed_data/YorkUrbanDB/P1020817", "P1020817.jpg")
image = skimage.io.imread(iname)[:, :, 0:3]

In [61]:
image.shape

(480, 640, 3)

In [62]:
org_segs

tensor([[315.,  87., 412., 135.],
        [316., 133., 409., 165.],
        [226., 328., 315., 307.],
        [123., 140., 277., 168.],
        [123., 310., 263., 288.],
        [122., 189., 263., 201.],
        [124., 291., 261., 273.],
        [205., 116., 223., 122.],
        [206.,  83., 224.,  89.],
        [170.,   6., 342.,  95.],
        [ 38.,   9., 279., 102.],
        [238., 257., 336., 253.],
        [411., 138., 411., 164.],
        [358., 179., 356., 271.],
        [486., 177., 484., 297.],
        [249., 287., 249., 348.],
        [294., 272., 294., 326.],
        [328., 261., 328., 307.],
        [237., 260., 236., 320.],
        [201., 301., 201., 371.],
        [186., 300., 186., 369.],
        [277., 169., 276., 286.],
        [121., 140., 121., 308.],
        [265., 172., 265., 285.],
        [175., 159., 174., 283.],
        [225., 168., 223., 277.],
        [224.,  90., 224., 121.],
        [204., 116., 204.,  84.],
        [282., 257., 281., 313.],
        [317.,

In [63]:
org_segs = org_segs.reshape(-1,4)

In [64]:
org_segs.shape

torch.Size([69, 4])

In [65]:
num_segs = len(org_segs)

In [66]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[0, 640, 480, 0])
for i in range(num_segs):
    plt.plot(
        (org_segs[i, 0], org_segs[i, 2]),
        (org_segs[i, 1], org_segs[i, 3]),
        c="r",
        alpha=1.0,
    )                                 
plt.xlim(0, 640)
plt.ylim(480, 0)
plt.axis('off')
plt.savefig('yud_seg_ortho_0.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

In [67]:
ortho_vp

tensor([[ 0.4032,  0.0076, -0.9151],
        [ 0.0114,  0.9998,  0.0133],
        [ 0.9151, -0.0158,  0.4030]], dtype=torch.float64)

In [68]:
vpts_x = ortho_vp[0]
vpts_y = -ortho_vp[1]
vpts_z = ortho_vp[2]

In [69]:
focal_length = 320.

In [70]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[-320/focal_length, 320/focal_length, 240/focal_length, -240/focal_length])
for i in range(num_segs):
    plt.plot((0, vpts_x[0]), (0, vpts_x[1]), 'r-', alpha=1.0)
    plt.plot((0, vpts_y[0]), (0, vpts_y[1]), 'g-', alpha=1.0)
    plt.plot((0, vpts_z[0]), (0, vpts_z[1]), 'b-', alpha=1.0)                               
plt.xlim(-320/focal_length, 320/focal_length)
plt.ylim(240/focal_length, -240/focal_length)
plt.axis('off')
plt.savefig('yud_vp_ortho_0.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

In [ ]:
vp = torch.tensor([[ 8.28081649e-01, -6.62657775e-04,  5.60607120e-01],
        [-1.26166510e-01, -9.74566085e-01,  1.85210576e-01],
        [-5.46225955e-01,  2.24099323e-01,  8.07103896e-01]])


In [ ]:
torch.dot(vp[0],vp[1])

In [207]:
iname = "/home/kmuvcl/dataset/data/processed_data/0003_0.png"
npz_file = "/home/kmuvcl/dataset/data/processed_data/0003_0.npz"

In [208]:
with np.load(iname.replace(".png",".npz"), allow_pickle=True) as npz:
            vpts = npz["vpts"]
            org_segs = npz['line_segments']
            vp_inds = npz['line_vp_inds']

In [212]:
K

matrix([[674.91797516,   0.        , 307.55130528],
        [  0.        , 674.91797516, 251.45424496],
        [  0.        ,   0.        ,   1.        ]])

In [209]:
vpts

array([[ 0.4031716 ,  0.00758039, -0.915093  ],
       [-0.01141125, -0.9998463 , -0.01331003],
       [ 0.91505325, -0.01580858,  0.40302313]])

In [213]:
in_vpts = K*vpts

In [214]:
in_vpts

matrix([[ 5.53533581e+02,  2.54193757e-01, -4.93662424e+02],
        [ 2.22392368e+02, -6.78789376e+02,  9.23586971e+01],
        [ 9.15053247e-01, -1.58085790e-02,  4.03023132e-01]])

In [215]:
num_vp = in_vpts.shape[1]
tvp_list = []
for vi in range(num_vp):
    in_vpts[:, vi] /= in_vpts[2, vi]

    tVP = np.array(in_vpts[:, vi])[:, 0]
    tVP /= tVP[2]

    tvp_list += [tVP]

In [216]:
true_vps = np.vstack(tvp_list)

In [218]:
vp = true_vps

In [219]:
vp

array([[ 6.04919531e+02,  2.43037625e+02,  1.00000000e+00],
       [-1.60794817e+01,  4.29380385e+04,  1.00000000e+00],
       [-1.22489849e+03,  2.29164755e+02,  1.00000000e+00]])

In [220]:
vp[:,0] /= vp[:,2]
vp[:,1] /= vp[:,2]
vp[:,2] /= vp[:,2]

In [221]:
vp

array([[ 6.04919531e+02,  2.43037625e+02,  1.00000000e+00],
       [-1.60794817e+01,  4.29380385e+04,  1.00000000e+00],
       [-1.22489849e+03,  2.29164755e+02,  1.00000000e+00]])

In [222]:
vps_pixel = vp[:, 0:2]
vps_homo = np.concatenate((vps_pixel, np.ones((len(vps_pixel),1), dtype=np.float32)), axis=-1)
vps_homo[:, 0] -= 320
vps_homo[:, 1] -= 240
vps_homo[:, 1] *= -1
vps_homo[:, 0:2] /= 320.
vps_homo /= np.linalg.norm(vps_homo, axis=1, keepdims=True)

In [223]:
vps_homo

array([[ 0.66496631, -0.00708943,  0.74683971],
       [-0.00787061, -0.99994094,  0.00749405],
       [-0.9791913 ,  0.00686762,  0.20282317]])

In [226]:
torch.dot(torch.tensor(vps_homo[1]),torch.tensor(vps_homo[2]))

tensor(0.0024, dtype=torch.float64)

In [227]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[-320/focal_length, 320/focal_length, 240/focal_length, -240/focal_length])
for i in range(num_segs):
    plt.plot((0, vps_homo[0][0]), (0, vps_homo[0][1]), 'r-', alpha=1.0)
    plt.plot((0, vps_homo[1][0]), (0, vps_homo[1][1]), 'g-', alpha=1.0)
    plt.plot((0, vps_homo[2][0]), (0, vps_homo[2][1]), 'b-', alpha=1.0)                               
plt.xlim(-320/focal_length, 320/focal_length)
plt.ylim(240/focal_length, -240/focal_length)
plt.axis('off')
plt.savefig('yud_vp_ortho_homo.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

In [180]:
vpts = torch.tensor(vpts)

In [181]:
vpts

tensor([[ 5.5353e+02,  2.5419e-01, -4.9366e+02],
        [ 2.2239e+02, -6.7879e+02,  9.2359e+01],
        [ 9.1505e-01, -1.5809e-02,  4.0302e-01]], dtype=torch.float64)

In [182]:
vpts_x = vpts[0]/vpts[0][2]
vpts_y = -vpts[1]/vpts[1][2]
vpts_z = vpts[2]/vpts[2][2]
focal_length = 320.

In [183]:
# vpts_x[0] = (vpts_x[0] - 320)/320
# vpts_x[1] = -(vpts_x[1] - 240)/320

# vpts_y[0] = (vpts_y[0] - 320)/320
# vpts_y[1] = -(vpts_y[1] - 240)/320

# vpts_z[0] = (vpts_z[0] - 320)/320
# vpts_z[1] = -(vpts_z[1] - 240)/320



In [184]:
vpts_z

tensor([ 2.2705, -0.0392,  1.0000], dtype=torch.float64)

In [185]:
image = skimage.io.imread(iname)[:, :, 0:3]

In [186]:
image.shape

(480, 640, 3)

In [187]:
vpts = torch.tensor(vpts)

/tmp/ipykernel_646654/4071118012.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vpts = torch.tensor(vpts)


In [188]:
torch.dot(vpts[1],vpts[2])

tensor(251.4542, dtype=torch.float64)

In [189]:
org_segs.shape

(69, 4)

In [190]:
num_segs=len(org_segs)

In [191]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[0, 640, 480, 0])
for i in range(num_segs):
    plt.plot(
        (org_segs[i, 0], org_segs[i, 2]),
        (org_segs[i, 1], org_segs[i, 3]),
        c="r",
        alpha=1.0,
    )                                 
plt.xlim(0, 640)
plt.ylim(480, 0)
plt.axis('off')
plt.savefig('yud_seg_ortho_in.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

In [192]:
plt.figure(figsize=(5,5))
plt.imshow(image, extent=[-320/focal_length, 320/focal_length, 240/focal_length, -240/focal_length])
for i in range(num_segs):
    plt.plot((0, vpts_x[0]), (0, vpts_x[1]), 'r-', alpha=1.0)
    plt.plot((0, vpts_y[0]), (0, vpts_y[1]), 'g-', alpha=1.0)
    plt.plot((0, vpts_z[0]), (0, vpts_z[1]), 'b-', alpha=1.0)                               
plt.xlim(-320/focal_length, 320/focal_length)
plt.ylim(240/focal_length, -240/focal_length)
plt.axis('off')
plt.savefig('yud_vp_ortho_in.jpg',  
            pad_inches=0, bbox_inches='tight')
plt.close('all')

TypeError: unsupported operand type(s) for *: 'matrix' and 'Tensor'